In [2]:
pip install torch torchvision transformers huggingface_hub flask  numpy pillow


Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from PIL import Image
import numpy as np
import requests

# Charger le modèle pré-entraîné
model_name = "dima806/facial_emotions_image_detection"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)
# model.eval()


2025-03-11 10:05:58.374755: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:

def analyze_image(image_path):
    # Charger l'image
    image = Image.open(image_path).convert("RGB")

    # Transformer l'image en format compatible avec le modèle
    inputs = processor(
        images=image,
        return_tensors="pt",
        do_resize=True,
        size={"height": 224, "width": 224},
        do_normalize=True
    )

    # Passer l'image dans le modèle
    with torch.no_grad():
        outputs = model(**inputs)

    # Récupérer les prédictions
    logits = outputs.logits
    probas = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class_idx = torch.argmax(probas, dim=-1).item()

    # Obtenir les labels du modèle
    labels = model.config.id2label
    predicted_emotion = labels[predicted_class_idx]

    return {
        "emotion": predicted_emotion,
        "confidence": f"{probas[0][predicted_class_idx].item():.2%}"
    }

image_path = "/Users/barrysalam/Desktop/hackathon-ipssi-mia-web/models/images/triste.jpg"
result = analyze_image(image_path)

# Afficher les résultats
print(f"Émotion détectée : {result['emotion']}")
print(f"Niveau de confiance : {result['confidence']}")

Émotion détectée : sad
Niveau de confiance : 62.22%
